## Implicit Library

In [24]:
import implicit
#import h5py
import pandas as pd
import numpy as np
import random
import statistics
from implicit.evaluation import train_test_split
from sklearn.metrics import ndcg_score
from numpy.random import permutation
from implicit.nearest_neighbours import (
    normalize,
    bm25_weight
)
import scipy
from scipy.sparse import csr_matrix
import scipy.sparse
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import precision_at_k,mean_average_precision_at_k, ndcg_at_k
from random import shuffle
import itertools
import copy


## Preprocessing

In [25]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [26]:

data = df

In [27]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [28]:
data = df

In [29]:
matrix = pd.read_csv("data/origin_binary_matrix.csv")

In [30]:
matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
data_skills  = matrix
data_skills 

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
data_skills_csr = scipy.sparse.csr_matrix(data_skills.values)

In [33]:
data = df
data.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


# GridsearchCV

In [34]:
param_grid_bm25  = {'K1': [3,100], #2,20,50,100
                    'B': [0.6,0.8] # 0.4,0.8,1
                    }

# param_grid_bm25  = {'K1': [2,50,100], #2,20,50,100
#                     'B': [ 0.4,0.8,1] # 0.4,0.8,1
#                     }

In [35]:
def normalizer (data, param_grid):
    data_representations = ['Raw','Normalized', 'tfidf', 'bm25']    
    df_list = []
    data_list = []
    for representation in data_representations:
        df = pd.DataFrame(columns = ['data_representations', 'K1', 'B'])


        if representation == 'Normalized':
            data_representations = implicit.nearest_neighbours.normalize(data)
            data_list.append(data_representations)
            df["data_representations"] = [representation]
            df["K1"] = np.nan
            df["B"] = np.nan  
            df_list.append(df)

        elif representation == 'tfidf':
            data_representations = implicit.nearest_neighbours.tfidf_weight(data)
            data_list.append(data_representations)
            df["data_representations"] = [representation]
            df["K1"] = np.nan
            df["B"] = np.nan  
            df_list.append(df)

        elif representation == 'bm25':
            keys, values = zip(*param_grid.items())
            for c,v in enumerate(itertools.product(*values)):
                params = dict(zip(keys, v))
                
                #############
                # Initiate df for the parameter combination
                #############
                df = pd.DataFrame()
                df = pd.DataFrame(params, index =  [c])
                df["data_representations"] = [representation]
                df_list.append(df)

                #############
                # Add key value pair to the dictonary and initiate the bm25_weight function for every parameter combination
                #############
                params['X'] = data
                data_representations = implicit.nearest_neighbours.bm25_weight(**params)
                data_list.append(data_representations)

        else: 
            df["data_representations"] = [representation]
            df["K1"] = np.nan
            df["B"] = np.nan  
            df_list.append(df)
            data_list.append(data)
    return data_list, pd.concat(df_list).reset_index().drop(columns = ['index'])

In [36]:
daten, df = normalizer(data_skills_csr, param_grid_bm25)
df 


,data_representations,K1,B
0,Raw,NaN,NaN
1,Normalized,NaN,NaN
2,tfidf,NaN,NaN
3,bm25,3.0,0.6
4,bm25,3.0,0.8
5,bm25,100.0,0.6
6,bm25,100.0,0.8


In [37]:
def cross_validation_implicit_gs(data, algorithm, param_grid):
    df_list = []

    data_list, df = normalizer(data, param_grid_bm25)

    df_result = pd.DataFrame()
    for b,d in enumerate(data_list):
        df1 = pd.DataFrame()
        df1 = df.loc[b].to_frame().T
        data_rep = d

        keys, values = zip(*param_grid.items())
        for v in itertools.product(*values):
            
            #############
            # This for loop produces every possible combination of the hyperparameters within a dictonary
            #############
            params = dict(zip(keys, v))
            this_model = copy.deepcopy(algorithm)
            df2 = pd.DataFrame()
            df2 = pd.DataFrame(params, index =  [b])
    
            

            for k, v in params.items():
                #############
                # This loop unpacks the diconary by each parameter and initiates the model for each hpyerparameter of the corresponding dictonary
                #############
                setattr(this_model, k, v)

            ############
            # lists to store the results after each train_test_split under different random seed (c.p)
            ############
            p_train_test_results = []
            map_train_test_results = []
            ndcg_train_test_results = []


            for r_seed in range(0,5):
                #############
                # Initiate the train_test_split
                #############
                train_mat, test_mat = implicit.evaluation.train_test_split(data_rep, train_percentage =  0.8, random_state = r_seed)
                this_model.fit(train_mat,show_progress=False)

                #############
                # measure evaluation metrixs for each possible hyperparameter combination
                #############
                p_at_k =  precision_at_k(this_model, train_user_items=train_mat, test_user_items=test_mat, K=5 ,show_progress=False)
                map_at_k = mean_average_precision_at_k(this_model, train_user_items=train_mat, test_user_items=test_mat, K=5,show_progress=False)
                var_ndcg_at_k = ndcg_at_k(this_model, train_user_items=train_mat, test_user_items=test_mat, K=5,show_progress=False)


                #############
                # To list
                #############
                p_train_test_results.append(p_at_k)
                map_train_test_results.append(map_at_k)
                ndcg_train_test_results.append(var_ndcg_at_k)

            #############
            # Create columns to store the scores for each hyperparameter combination after 5 different train_test_splits
            #############
            df2["precision_at_k"] = np.mean(p_train_test_results)
            df2["map_at_k"]  = np.mean(map_train_test_results)
            df2["ndcg_at_k"] = np.mean(ndcg_train_test_results)

            df_result = df1.merge(df2, left_index=True, right_index=True)
            df_list.append(df_result)
    
    return pd.concat(df_list).reset_index().drop(columns = ['index'])

In [38]:
# als_grid = {
#               'factors': [1,2,3,4,5,30,40,50],
#               'regularization': [0.005,0.01,0.02],
#               'iterations' : [15,30,45], 
#               'alpha': [1,2] 
#               }

## ALS

In [39]:
# results_als= cross_validation_implicit_gs(data_skills_csr, algorithm = implicit.als.AlternatingLeastSquares(random_state = 42),param_grid =  als_grid)
# results_als

In [40]:
# results_als.to_csv("results_als.csv")

In [4]:
results_als = pd.read_csv("results_als.csv")

## LMF


In [20]:
# lmf_grid = {'factors': [1,2,3,4,5,30,40,50],
#               'regularization': [0.005,0.01,0.02,0.025,0.03],
#               'iterations' : [15,30,45],
#               'alpha': [1,2,4] 
#               }

In [21]:
# results_lmf= cross_validation_implicit_gs(data_skills_csr, algorithm = implicit.lmf.LogisticMatrixFactorization(random_state = 42),param_grid =  lmf_grid)
# results_lmf

In [22]:
# results_lmf.to_csv('results_lmf.csv')

In [23]:
results_lmf = pd.read_csv('results_lmf.csv')

## Get the best model according to the ndcg score 

In [41]:
best_k1 = results_als.iloc[results_als.ndcg_at_k.idxmax()]["K1"]
best_B = results_als.iloc[results_als.ndcg_at_k.idxmax()]["B"]
best_data_representation = results_als.iloc[results_als.ndcg_at_k.idxmax()]["data_representations"]
best_factors  = results_als.iloc[results_als.ndcg_at_k.idxmax()]["factors"]
best_regularization = results_als.iloc[results_als.ndcg_at_k.idxmax()]["regularization"]
best_iterations = results_als.iloc[results_als.ndcg_at_k.idxmax()]["iterations"]
best_alpha  = results_als.iloc[results_als.ndcg_at_k.idxmax()]["alpha"]
results_als.iloc[results_als.ndcg_at_k.idxmax()]

Unnamed: 0                    86
data_representations         Raw
K1                           NaN
B                            NaN
factors                        5
regularization              0.02
iterations                    30
alpha                          1
precision_at_k          0.533082
map_at_k                0.430811
ndcg_at_k               0.521339
Name: 86, dtype: object

In [42]:
best_grid_bm25  = { 'X': data_skills_csr,
                    'K1': best_k1, 
                    'B': best_B 
                    }

In [43]:
data_representation_dict  = {'Raw':data_skills_csr
                            ,'Normalized':  implicit.nearest_neighbours.normalize(data_skills_csr)
                            , 'tfidf':implicit.nearest_neighbours.tfidf_weight(data_skills_csr)
                            , 'bm25':implicit.nearest_neighbours.bm25_weight(**best_grid_bm25)
                            }    

In [44]:
data_best = data_representation_dict[best_data_representation]
data_best = data_best.tocsr()

## Initiate that model

In [45]:
train_mat, test_mat = implicit.evaluation.train_test_split(data_best, train_percentage =  0.8, random_state = 42)

In [52]:
model_best = implicit.als.AlternatingLeastSquares(factors = best_factors, regularization = best_regularization, iterations = best_iterations, random_state = 42)
model_best.fit(train_mat, show_progress=False)

# Performance

## Baseline

In [30]:
most_common_skills = matrix.sum(axis=0).nlargest(5)	
for i in most_common_skills.index:
    print(i)

Englisch
MySQL
SCRUM
GIT
HTML


In [31]:
p_at_5_baseline_list  = []

for i in range(0,train_mat.shape[0]):
    user_series = pd.DataFrame.sparse.from_spmatrix(train_mat, columns=matrix.columns).loc[i]
    user_know_skills = user_series[user_series > 0]
    user_known_skills_vs_most_common = user_know_skills.index.isin(most_common_skills.index)
    matches = user_known_skills_vs_most_common.sum()
    results =matches/5
    p_at_5_baseline_list.append(results)
 
p_at_5_baseline = statistics.mean(p_at_5_baseline_list)
p_at_5_baseline


0.5475247524752476

## Model

In [54]:
implicit.evaluation.precision_at_k(model_best, train_user_items=train_mat, test_user_items=test_mat, K=5,show_progress=False)

0.5416913069189828

Wie aussagekräftig ist p at k ? Ein recommender, der immmer die häufigsten Skills vorschlägt ist nicht zielführend, hat haben einen hohen p_at_5 score

### Do some recommendations

In [55]:
data_categories = data[["skill","category"]].drop_duplicates()
data_categories.head(5)

,skill,category
0,Windows NT/2000/XP,Betriebssystem
1,MAC OS X,Betriebssystem
2,Windows 7,Betriebssystem
3,Windows 8,Betriebssystem
4,iOS,Betriebssystem


In [ ]:
# Id and skills
dict_skills_id = {}
for c,skill in enumerate(matrix.columns):
        dict_skills_id[c] = skill


In [71]:
def get_key_from_value(d, val):
    keys_list = []
    for i in val:
        keys = [k for k, v in d.items() if v == i]
        keys_list.append(keys)
    return keys_list

In [82]:
userid= [0,1]

def do_recommendations (userid, user_items_mat_train,full_data_mat ,item_to_exclude = None ,n = 10):
    df_list = []
    item_to_exclude  = get_key_from_value(dict_skills_id,item_to_exclude)
    for u in userid:
        
        
        #############
        # Do recommendations for every user in the list. The known items are in the matrix the model was trained on
        #############
        skill_ids, scores  = model_best.recommend(u, user_items= user_items_mat_train[u], N = n , filter_already_liked_items=True, filter_items = item_to_exclude)
        recommendations_df = pd.DataFrame({ "skill": matrix.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, user_items_mat_train.tocsr()[u].indices)})
        recommendations_df["user_id"] = [u] * len(recommendations_df)
        recommendations_df = recommendations_df.rename(columns = {"already_liked": "already_liked_in_train"})

        #############
        # Get all known skills for each user by indexing the full data matrix
        #############
        data_best_user_id = pd.DataFrame.sparse.from_spmatrix(data = full_data_mat.tocsr()[u],columns=matrix.columns).T.reset_index().rename(columns = {"index": "skill",0: "rating"}).sort_values(by = "rating", ascending = False)
        data_best_user_id["user_id"] = [u] * len(data_best_user_id)
      
        #############
        # Merge both df in order to see, if the recommended skills are the skills, which the user already knows, but were left out in the trainings data
        #############
        recommendations_df = recommendations_df.merge(data_best_user_id, on = ["skill","user_id"], how = "left").rename(columns = {"rating": "rating_in_original"})
        

        #############
        # Merge the category information to the recommendations
        #############
        recommendations_df = recommendations_df.merge(data_categories, on = "skill", how = "left")
        recommendations_df.set_index("user_id", inplace = True)
        df_list.append(recommendations_df)
    return pd.concat(df_list)

In [84]:
recommendations_df = do_recommendations(userid, train_mat,data_best,n = 10,item_to_exclude= ["MySQL"])
recommendations_df

,skill,score,already_liked_in_train,rating_in_original,category
user_id,,,,,
0,SCRUM,0.902770,False,1,Projektmanagement / Vorgehensmodelle
0,Unit Tests (Komponententests),0.783137,False,1,Methoden und Praktiken
0,Dependency Injection,0.666445,False,1,Methoden und Praktiken
0,REST,0.658121,False,1,Standards
0,Dokumentation,0.643176,False,0,Einsatzfelder / Erfahrungen / Schwerpunkte
0,CSS,0.632713,False,0,Webentwicklung
0,CSS,0.632713,False,0,Programmiersprachen / Scriptsprachen
0,Softwaredesign/-architektur,0.626180,False,0,Einsatzfelder / Erfahrungen / Schwerpunkte
0,XML/XSL,0.588169,False,0,Standards


## New user

In [36]:
skill_ids, scores  = model_best.recommend(0, user_items= train_mat[0], N = 5 , filter_already_liked_items=True, recalculate_user=True)
recommendations_df = pd.DataFrame({ "skill": matrix.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, train_mat.tocsr()[0].indices)})
recommendations_df = recommendations_df.rename(columns = {"already_liked": "already_liked_in_train"})
recommendations_df

,skill,score,already_liked_in_train
0,SCRUM,0.892121,False
1,Unit Tests (Komponententests),0.774384,False
2,MySQL,0.772703,False
3,Dependency Injection,0.659050,False
4,REST,0.650433,False


In [37]:
# x.toarray()
# >> array([[ 1.,  1.,  1.,  1.,  1.],
#           [ 1.,  1.,  1.,  1.,  1.],
#           [ 1.,  1.,  1.,  1.,  1.]])
# # reshape is not implemented for csr_matrix but you can cheat and do it  yourself.
# x._shape = (4,5)
# # Update indptr to let it know we added a row with nothing in it. So just append the last
# # value in indptr to the end.
# # note that you are still copying the indptr array
# x.indptr = np.hstack((x.indptr,x.indptr[-1]))
# x.toarray()
# array([[ 1.,  1.,  1.,  1.,  1.],
#        [ 1.,  1.,  1.,  1.,  1.],
#        [ 1.,  1.,  1.,  1.,  1.],
#        [ 0.,  0.,  0.,  0.,  0.]])

In [38]:
new_id = 404
new_skills = {
                "Windows 11":1,
                "Englisch":1,
                "Chinesisch (Mandarin)":1,
                "Python":1,
                "MATLAB":1,
                "Java":1,
                "MySQL":1,
                "SQLBase":1,
                "Microsoft SQL Server":1,
                "Google Cloud Platform":1,
                "MongoDB":1,
                "JSON":1,
                "Docker":1,
                "Statische Codeanalyse": 1,
                "Power BI":1,
                "Postman":1,
                "PowerShell":1,
                "Github actions":1
            }

In [39]:
new_user_df = pd.DataFrame(new_skills, index = [new_id])
new_user_df

,Windows 11,Englisch,Chinesisch (Mandarin),Python,MATLAB,Java,MySQL,SQLBase,Microsoft SQL Server,Google Cloud Platform,MongoDB,JSON,Docker,Statische Codeanalyse,Power BI,Postman,PowerShell,Github actions
404,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [40]:
new_user_matrix = pd.concat([matrix,new_user_df]).sort_index().fillna(0)
new_user_matrix.tail(5)

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
403,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ebenfalls normalizen

In [41]:
new_user_matrix_csr = scipy.sparse.csr_matrix(new_user_matrix.values)
new_user_matrix_csr


<405x735 sparse matrix of type '<class 'numpy.float64'>'
	with 24187 stored elements in Compressed Sparse Row format>

In [42]:
skill_ids, scores  = model_best.recommend(404, user_items= new_user_matrix_csr[404], N = 5 , filter_already_liked_items=True, recalculate_user=True)
recommendations_df = pd.DataFrame({ "skill": new_user_matrix.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, new_user_matrix_csr.tocsr()[404].indices)})
recommendations_df

,skill,score,already_liked
0,Test Driven Development (TDD),0.191458,False
1,Unit Tests (Komponententests),0.190572,False
2,GIT,0.189136,False
3,SCRUM,0.184054,False
4,REST,0.178197,False


### Simularities

### Items

In [43]:
# Id and skills
dict_skills_id = {}
for c,skill in enumerate(matrix.columns):
        dict_skills_id[c] = skill

In [44]:
skill_id, sim = model_best.similar_items(0, N=10)
# display the results using pandas for nicer formatting
simularity_df  = pd.DataFrame({"skill": matrix.columns[skill_id], "score": sim})
simularity_df.merge(data_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,category
0,.NET Compact Framework,1.000000,.NET Frameworks und Tools
1,MonoDevelop,0.989928,Umgebungen
2,Cruise Control.NET,0.987770,"CI/CD, Build- und Versionskontrollsysteme"
3,ASP Generalist,0.983801,.NET Frameworks und Tools
4,Mono,0.980985,.NET Frameworks und Tools
5,VB.NET,0.979146,Programmiersprachen / Scriptsprachen
6,BizTalk,0.974194,.NET Frameworks und Tools
7,Exact,0.967203,ERP
8,AS400,0.961355,Betriebssystem
9,Sharepoint Entwicklung,0.952586,.NET Frameworks und Tools


### Users

In [45]:
new_skills = {
                "Windows 11":1,
                "Englisch":1,
                "Chinesisch (Mandarin)":1,
                "Python":1,
                "MATLAB":1,
                "Java":1,
                "MySQL":1,
                "SQLBase":1,
                "Microsoft SQL Server":1,
                "Google Cloud Platform":1,
                "MongoDB":1,
                "JSON":1,
                "Docker":1,
                "Statische Codeanalyse": 1,
                "Power BI":1,
                "Postman":1,
                "PowerShell":1,
                "Github actions":1
            }

In [46]:
new_user_id_list = list(new_skills.keys())

In [47]:
user_ids_, sim = model_best.similar_users(0, N=10)
# display the results using pandas for nicer formatting
simularity_df  = pd.DataFrame({"user_ids": matrix.index[user_ids_], "score": sim})
simularity_df

,user_ids,score
0,0,1.000000
1,362,0.997664
2,341,0.991820
3,276,0.991532
4,269,0.989979
5,256,0.989797
6,55,0.989418
7,154,0.988434
8,86,0.987851
9,67,0.985626


In [48]:
## Compare the mst similar user to the new user
matrix[new_user_id_list].iloc[241].to_frame().T

,Windows 11,Englisch,Chinesisch (Mandarin),Python,MATLAB,Java,MySQL,SQLBase,Microsoft SQL Server,Google Cloud Platform,MongoDB,JSON,Docker,Statische Codeanalyse,Power BI,Postman,PowerShell,Github actions
241,0,1,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0


# Recommend users

In [49]:
data_best_u = implicit.nearest_neighbours.bm25_weight(data_skills_csr.T, K1 = 100, B = 0.8)
data_best_u = data_best_u.tocsr()
data_best_u

<735x404 sparse matrix of type '<class 'numpy.float64'>'
	with 24169 stored elements in Compressed Sparse Row format>

In [50]:
train_mat_u, test_mat_u = implicit.evaluation.train_test_split(data_best_u, train_percentage =  0.8, random_state = 42)

In [51]:
model_best_u = implicit.als.AlternatingLeastSquares(factors = 30, regularization = 0.01, iterations = 15, alpha = 1, random_state = 42)
model_best_u.fit(train_mat_u)

100%|██████████| 15/15 [00:00<00:00, 21.08it/s]


In [52]:
id = 0 
user_ids, scores  = model_best_u.recommend(id, user_items= train_mat_u[id], N = 10 , filter_already_liked_items=False)
user_ids, scores 
recommendations_df_u = pd.DataFrame({ "user": user_ids, "score": scores, "already_liked": np.in1d(user_ids, train_mat_u.tocsr()[id].indices)})
recommendations_df_u

,user,score,already_liked
0,327,1.001292,True
1,386,0.901445,True
2,122,0.874448,True
3,95,0.809985,True
4,11,0.786734,True
5,179,0.731412,True
6,68,0.723188,False
7,329,0.713261,True
8,166,0.657552,True
9,141,0.630063,True


In [54]:
def do_recommendations_u (itemid, items_user_mat_train,full_data_mat ,n = 10):
    df_list = []
    for i in itemid:
        

        #############
        # Do recommendations for every user in the list. The known items are in the matrix the model was trained on
        #############
        user_ids, scores  = model_best_u.recommend(i, user_items= items_user_mat_train[i], N = n , filter_already_liked_items=True)
        recommendations_df = pd.DataFrame({ "user_ids": matrix.T.columns[user_ids], "score": scores, "already_liked": np.in1d(user_ids, items_user_mat_train.tocsr()[i].indices)})
        recommendations_df["skill_id"] = [i] * len(recommendations_df)
        recommendations_df = recommendations_df.rename(columns = {"already_liked": "already_liked_in_train"})

        #############
        # Get all known skills for each user by indexing the full data matrix
        #############
        data_best_skill_id =pd.DataFrame.sparse.from_spmatrix(data = full_data_mat.tocsr()[0]).T.reset_index().rename(columns = {"index": "user_ids",0: "rating"}).sort_values(by = "rating", ascending = False)
        data_best_skill_id["skill_id"] = [i] * len(data_best_skill_id)
      
        #############
        # Merge both df in order to see, if the recommended skills are the skills, which the user already knows, but were left out in the trainings data
        #############
        recommendations_df = recommendations_df.merge(data_best_skill_id, on = ["user_ids","skill_id"], how = "left").rename(columns = {"rating": "rating_in_original"})
        recommendations_df.set_index("skill_id", inplace = True)
        recommendations_df = recommendations_df.rename(index=dict_skills_id)
        df_list.append(recommendations_df)
    return pd.concat(df_list)

In [55]:
results_test = do_recommendations_u([0], train_mat_u, data_best_u, n = 5)
results_test

,user_ids,score,already_liked_in_train,rating_in_original
skill_id,,,,
.NET Compact Framework,68,0.723188,False,2.076634
.NET Compact Framework,205,0.625273,False,0.000000
.NET Compact Framework,187,0.584479,False,2.266041
.NET Compact Framework,374,0.582255,False,0.000000
.NET Compact Framework,376,0.503813,False,0.000000


In [56]:
matrix.loc[205]

.NET Compact Framework    0
.NET Core                 0
.NET Framework            1
3D-Modellierung           0
ABAP                      0
                         ..
visual paradigm           0
vnc                       1
vs code                   1
xHTML                     1
xtCommerce                0
Name: 205, Length: 735, dtype: int64

# Drop Skills

### Drop skills with a relative min support of =< 5%

In [82]:
rare_skills = data_skills.sum()/len(data_skills) #data_skills.sum()[data_skills.sum() > 20].index.tolist()
rare_skills = rare_skills[rare_skills > 0.05].index.tolist()

In [83]:
data_skills_rare  = data_skills.loc[:, data_skills.columns.isin(rare_skills)]
data_skills_rare

,.NET Compact Framework,.NET Framework,3D-Modellierung,ADO.NET,ASP Generalist,ASP.NET,ASP.NET MVC-Framework,ASP.NET WebAPI,AWS,Access,...,gpg,iOS,jQuery,nginx,npm,prototype,samba,vnc,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,1,0,1,1,1,0,0,...,0,1,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
402,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
data_skills_rare_csr = scipy.sparse.csr_matrix(data_skills_rare.values)


In [90]:
test = pd.DataFrame({"A":[1,2,3], "B":[4,5,6]})
test["C"]= len(test) 
test

,A,B,C
0,1,4,3
1,2,5,3
2,3,6,3


In [91]:
als_grid = {
              'factors': [1,2,3,4,5,6],
              'regularization': [0.005,0.01,0.02,0.03],
              'iterations' : [15,30,45], 
              'alpha': [1,2] 
              }

In [92]:
results_list = []
for i in [data_skills_csr, data_skills_rare_csr]:
    number_cols = i.shape[1]
    results = cross_validation_implicit_gs(i, algorithm = implicit.als.AlternatingLeastSquares(random_state = 42),param_grid =  als_grid)
    results["number_of_skills"] = number_cols
    results_list.append(results)

results_als = pd.concat(results_list)

In [93]:
results_als

,data_representations,K1,B,factors,regularization,iterations,alpha,precision_at_k,map_at_k,ndcg_at_k,number_of_skills
0,Raw,NaN,NaN,1,0.005,15,1,0.459071,0.357035,0.444635,735
1,Raw,NaN,NaN,1,0.005,15,2,0.465052,0.361346,0.450147,735
2,Raw,NaN,NaN,1,0.005,30,1,0.459071,0.357035,0.444635,735
3,Raw,NaN,NaN,1,0.005,30,2,0.465052,0.361346,0.450147,735
4,Raw,NaN,NaN,1,0.005,45,1,0.459071,0.357035,0.444635,735
...,...,...,...,...,...,...,...,...,...,...,...
1003,bm25,100.0,0.8,6,0.030,15,2,0.495536,0.376542,0.479516,270
1004,bm25,100.0,0.8,6,0.030,30,1,0.496781,0.377012,0.481202,270
1005,bm25,100.0,0.8,6,0.030,30,2,0.496277,0.376914,0.480229,270
1006,bm25,100.0,0.8,6,0.030,45,1,0.496776,0.376770,0.481027,270


In [ ]:
results_als.to_csv("results_als_2.csv")

In [111]:
data_list = [data_skills_csr, data_skills_rare_csr]
results_als_rare  = cross_validation_implicit_gs(data_skills_rare, algorithm = implicit.als.AlternatingLeastSquares(random_state = 42),param_grid =  als_grid)
results_als_rare

,data_representations,K1,B,factors,regularization,iterations,alpha,precision_at_k,map_at_k,ndcg_at_k
0,Raw,NaN,NaN,1,0.01,15,1,0.484414,0.377373,0.468191
1,Raw,NaN,NaN,1,0.01,15,2,0.486008,0.378640,0.471007
2,Raw,NaN,NaN,1,0.01,30,1,0.484414,0.377373,0.468191
3,Raw,NaN,NaN,1,0.01,30,2,0.486008,0.378640,0.471007
4,Raw,NaN,NaN,2,0.01,15,1,0.516257,0.404838,0.494659
...,...,...,...,...,...,...,...,...,...,...
107,bm25,100.0,0.8,30,0.01,30,2,0.449446,0.331172,0.444711
108,bm25,100.0,0.8,40,0.01,15,1,0.416874,0.299683,0.413978
109,bm25,100.0,0.8,40,0.01,15,2,0.427544,0.311688,0.426726
110,bm25,100.0,0.8,40,0.01,30,1,0.412702,0.296715,0.410715


In [113]:
results_als_rare.iloc[results_als_rare.ndcg_at_k.idxmax()]

data_representations         Raw
K1                           NaN
B                            NaN
factors                        5
regularization              0.01
iterations                    30
alpha                          2
precision_at_k          0.563731
map_at_k                0.450729
ndcg_at_k               0.543477
Name: 19, dtype: object

In [7]:
results_als_2 = pd.read_csv("results_als_2.csv",index_col = 0)
results_als_2

,data_representations,K1,B,factors,regularization,iterations,alpha,precision_at_k,map_at_k,ndcg_at_k,number_of_skills
0,Raw,NaN,NaN,1,0.005,15,1,0.459071,0.357035,0.444635,735
1,Raw,NaN,NaN,1,0.005,15,2,0.465052,0.361346,0.450147,735
2,Raw,NaN,NaN,1,0.005,30,1,0.459071,0.357035,0.444635,735
3,Raw,NaN,NaN,1,0.005,30,2,0.465052,0.361346,0.450147,735
4,Raw,NaN,NaN,1,0.005,45,1,0.459071,0.357035,0.444635,735
...,...,...,...,...,...,...,...,...,...,...,...
1003,bm25,100.0,0.8,6,0.030,15,2,0.495536,0.376542,0.479516,270
1004,bm25,100.0,0.8,6,0.030,30,1,0.496781,0.377012,0.481202,270
1005,bm25,100.0,0.8,6,0.030,30,2,0.496277,0.376914,0.480229,270
1006,bm25,100.0,0.8,6,0.030,45,1,0.496776,0.376770,0.481027,270


In [8]:
results_als_2.iloc[results_als_2.ndcg_at_k.idxmax()]

data_representations         Raw
K1                           NaN
B                            NaN
factors                        6
regularization             0.005
iterations                    15
alpha                          1
precision_at_k          0.543289
map_at_k                0.436319
ndcg_at_k               0.528616
number_of_skills             735
Name: 120, dtype: object

# To Do's

* Skills droppen
* Skills aus recommendations entfernen
* NN Ansatz ausprobieren
* Epochen

### Preprocessing function


In [ ]:
pd.DataFrame.sparse.from_spmatrix(daten[2], columns=data_skills.columns)

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
1,0.0,0.00000,1.547068,0.000000,0.000000,2.823361,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
2,0.0,0.00000,0.000000,2.089392,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,1.291885,0.0
3,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
4,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
400,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
401,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
402,0.0,3.60352,1.547068,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0


### Single setup validation

In [ ]:
daten =  implicit.nearest_neighbours.bm25_weight(data_skills_csr, K1 = 20, B = 0.4)

In [ ]:
model_als = implicit.als.AlternatingLeastSquares(factors = 30, random_state= 42, alpha= 1, regularization= 0.01, iterations= 15)

ndcg_train_test_results = []
for r_seed in range(0,5):
    train_mat, test_mat = implicit.evaluation.train_test_split(daten, train_percentage =  0.8, random_state = r_seed)
    model_als.fit(train_mat)


    #############
    # measure evaluation metrixs for each possible hyperparameter combination
    #############
    var_ndcg_at_k = ndcg_at_k(model_als, train_user_items=train_mat, test_user_items=test_mat, K=5)
    ndcg_train_test_results.append(var_ndcg_at_k)

print(np.mean(ndcg_train_test_results))
    

100%|██████████| 374/374 [00:00<00:00, 4453.40it/s]

0.44710373923827795
